In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt 
from pandas import datetime
from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
from numba import jit
from scipy import stats

In [3]:
# Display and formatting
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 1200)
%matplotlib inline

In [4]:
import datetime
index = pd.date_range(start="2019-07-01", periods=101, freq='1s')
columns = ['s', 'res_price', 'inv', 'ask_spread', 'bid_spread', 
           'spread', 'bid action', 'ask_action', 'pnl']
df = pd.DataFrame(index=index, columns=columns)
df = df.fillna(0) # with 0s 

In [5]:
# Same initial setup
s0 = 100 # initial price
T = 1# time
sigma = 0.01 # volatility
dt = 0.01 # number of time steps - this creates 200 price points
q0 = 0 # time steps
gamma = 0.1 # risk aversion set to 0.1, can be parameterised based on pnl
k = 1.5 # set to 1.5
A = 140


In [6]:
# Create prices
white_noise = sigma * np.sqrt(dt) * np.random.choice([1, -1], int(T / dt))
price_process = s0 + np.cumsum(white_noise)
price_process = np.insert(price_process, 0, s0)
df['s'] = price_process

In [7]:
# Create step for time decay function
df['step'] = 1
df['step'] = df['step'].cumsum()

In [8]:
# Create inventory and flow (random) variables
inv = np.random.randint(-5,5,size=(101, 1))
df['inv'] = inv
flow = np.random.randint(-1,1,size=(101, 1))
df['flow'] = flow

In [9]:
# Original version with time decay function and no flow data
df['res_price'] = df['s'] - df['inv'] * gamma * sigma**2 * (T - df['step'] * dt)

df['spread'] = gamma * sigma**2 * (T - df['step'] * dt) + (2 / gamma) * np.log(1 + (gamma / k))
df['spread'] /= 2
df['ask_spread'] = np.where(df['res_price'] >= df['s'], df['spread'] + (df['res_price'] - df['s']), 
                            df['spread'] - (df['res_price'] - df['s']))
df['bid_spread'] = np.where(df['res_price'] >= df['s'], df['spread'] + (df['res_price'] - df['s']), 
                            df['spread'] - (df['res_price'] - df['s']))
df['time'] = (T - df['step'] * dt)

In [10]:
# Version with no time decay function 

df['res_price'] = df['s'] - (df['inv'] * gamma * sigma**2)

# this is the proper spread, need to estimate k
# df['spread'] = gamma * sigma**2 + (2 / gamma) * np.log(1 + (gamma / k))



# simple spread version
df['spread'] = df['s'] * sigma
df['spread'] /= 2
df['ask_spread'] = np.where(df['res_price'] >= df['s'], df['spread'] + (df['res_price'] - df['s']), 
                            df['spread'] - (df['res_price'] - df['s']))
df['bid_spread'] = np.where(df['res_price'] >= df['s'], df['spread'] + (df['res_price'] - df['s']), 
                            df['spread'] - (df['res_price'] - df['s']))

df['time'] = (T - df['step'] * dt)

df['bid_price'] = df['res_price'] - df['bid_spread']
df['ask_price'] = df['res_price'] + df['ask_spread']

In [74]:
sigma_freq = 1440
sigma_day = 0.02 # 2%
risk_premium = 3.5
gamma = .1
sigma_scaled = sigma_day / np.sqrt(sigma_freq)
spread = risk_premium * sigma_scaled * 9160
spread = spread/2
inv_adj = (10*100) * gamma * (sigma_day)**2/np.sqrt(sigma_freq)
reservation_price = 9160 - inv_adj





In [75]:
inv_adj*9160

9.655487789047452

In [76]:
spread

8.44855181541652

In [ ]:
df['spread'].iplot()